In [18]:
import pandas as pd

# Example: load 2023 season play-by-play

df = pd.read_csv("./play_by_play_2006.csv")

print(df.columns)
df.head()



/tmp/ipykernel_17150/527463854.py:5: DtypeWarning: Columns (45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,207,208,218,219,220,248,249,253,254,255,260,262,263,266,267,268,269,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./play_by_play_2006.csv")


Index(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       ...
       'out_of_bounds', 'home_opening_kickoff', 'qb_epa', 'xyac_epa',
       'xyac_mean_yardage', 'xyac_median_yardage', 'xyac_success', 'xyac_fd',
       'xpass', 'pass_oe'],
      dtype='object', length=372)


,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1,2006_01_ATL_CAR,2006091000,CAR,ATL,REG,1,NaN,NaN,NaN,...,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36,2006_01_ATL_CAR,2006091000,CAR,ATL,REG,1,ATL,away,CAR,...,0,0,0.892853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58,2006_01_ATL_CAR,2006091000,CAR,ATL,REG,1,ATL,away,CAR,...,0,0,0.119106,NaN,NaN,NaN,NaN,NaN,0.448167,-44.816697
3,80,2006_01_ATL_CAR,2006091000,CAR,ATL,REG,1,ATL,away,CAR,...,0,0,-0.851534,NaN,NaN,NaN,NaN,NaN,0.436872,-43.687209
4,102,2006_01_ATL_CAR,2006091000,CAR,ATL,REG,1,ATL,away,CAR,...,0,0,1.862235,0.218503,3.148713,1.0,0.998585,0.998585,0.966308,3.369230


In [23]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

# === CONFIGURATION ===
CSV_PATH = "./play_by_play_2006.csv"  # path to your CSV file

DB_CONFIG = {
    "dbname": "nfl_historical",
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",
    "port": 5432,
}

# === READ CSV ===
df = pd.read_csv(CSV_PATH)

# === CONNECT TO DATABASE ===
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()


# === CREATE TABLES IF NOT EXIST ===
cur.execute("""
CREATE TABLE IF NOT EXISTS games (
    id TEXT PRIMARY KEY,
    home_team TEXT NOT NULL,
    away_team TEXT NOT NULL,
    description TEXT NOT NULL,
    game_date date NOT NULL,
    season_type TEXT NOT NULL,
    week smallint NOT NULL
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS plays (
    id TEXT PRIMARY KEY,
    game_id TEXT NOT NULL REFERENCES games(id) ON DELETE CASCADE,
    posteam TEXT NOT NULL,
    posteam_type TEXT NOT NULL,
    defteam TEXT NOT NULL,
    total_home_score smallint,
    total_away_score smallint
);
""")

conn.commit()

# === INSERT INTO games ===
# Use UPSERT to avoid duplicates (in case same game appears multiple times)
games = df[["game_id", "home_team", "away_team", "desc", "game_date", "season_type", "week", ]].drop_duplicates(subset=["game_id"])

execute_values(
    cur,
    """
    INSERT INTO games (id, home_team, away_team, description, game_date, season_type, week)
    VALUES %s
    ON CONFLICT (id) DO UPDATE SET 
      home_team = EXCLUDED.home_team,
      away_team = EXCLUDED.away_team,
      game_date = EXCLUDED.game_date,
      season_type = EXCLUDED.season_type,
      week = EXCLUDED.week
    """,
    list(games.itertuples(index=False, name=None))
)

# === INSERT INTO plays ===
plays = df[["play_id", "game_id", "posteam", "posteam_type", "defteam", "total_home_score", "total_away_score"]].drop_duplicates(subset=["play_id"])

execute_values(
    cur,
    """
    INSERT INTO plays (id, game_id, posteam, posteam_type, defteam, total_home_score, total_away_score)
    VALUES %s
    ON CONFLICT (id) DO UPDATE SET
      game_id = EXCLUDED.game_id,
      posteam = EXCLUDED.posteam,
      posteam_type = EXCLUDED.posteam_type,
      defteam = EXCLUDED.defteam,
      total_home_score = EXCLUDED.total_home_score,
      total_away_score = EXCLUDED.total_away_score
    """,
    list(plays.itertuples(index=False, name=None))
)

# === COMMIT AND CLEANUP ===
conn.commit()
cur.close()
conn.close()

print("✅ Data import completed successfully.")


/tmp/ipykernel_17150/2443430144.py:17: DtypeWarning: Columns (45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,207,208,218,219,220,248,249,253,254,255,260,262,263,266,267,268,269,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_PATH)


✅ Data import completed successfully.
